# CNN from scratch

## Setup


In [1]:
import torch
import torch.nn as nn
from tensorboardX import SummaryWriter
from pa2_sample_code import get_datasets

train_data, eval_data = get_datasets()

# split train set into holdout_train and holdout_eval sets
holdout_train_len = int(len(train_data) * 0.8)
holdout_eval_len = len(train_data) - holdout_train_len
holdout_train_data, holdout_eval_data = torch.utils.data.random_split(train_data, [holdout_train_len, holdout_eval_len])

## Declare model

In [2]:
class CnnClassifier(nn.Module):
    def __init__(self, n_hidden):
        super(CnnClassifier, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=0),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
            nn.Sigmoid()
        )
        
        self.predictor = nn.Sequential(
            nn.Linear(in_features=32, out_features=n_hidden),
            nn.ReLU(),
            nn.Linear(in_features=n_hidden, out_features=47)
        )
        
        self.loss_func = nn.CrossEntropyLoss(reduction='sum')
        
    def forward(self, in_data):
        img_features = self.encoder(in_data).view(in_data.size(0), 32)
        logits = self.predictor(img_features)
        return logits

    def loss(self, logits, labels):
        return self.loss_func(logits, labels) / logits.size(0)
    
    def top_k_acc(self, logits, labels, k=1):
        _, k_labels_pred = torch.topk(logits, k=k, dim=1) # shape (n, k)
        k_labels = labels.unsqueeze(dim=1).expand(-1, k) # broadcast from (n) to (n, 1) to (n, k)
        # flatten tensors for comparison
        k_labels_pred_flat = k_labels_pred.reshape(1,-1).squeeze()
        k_labels_flat = k_labels.reshape(1,-1).squeeze()
        # get num_correct in float
        num_correct = k_labels_pred_flat.eq(k_labels_flat).sum(0).float().item()
        return num_correct / labels.size(0)
        

## Training

In [3]:
def train(model, loaders, optimizer, writer, num_epoch=10, device='cpu'):
    # for logging to tensorboard
    loss = {}
    top1 = {}
    top3 = {}
    def run_epoch(mode):
        epoch_loss = 0.0
        epoch_top1 = 0.0
        epoch_top3 = 0.0
        for i, batch in enumerate(loaders[mode], 0):
            in_data, labels = batch
            in_data, labels = in_data.to(device), labels.to(device)

            if mode == 'train':
                optimizer.zero_grad()

            logits = model(in_data)
            batch_loss = model.loss(logits, labels)
            batch_top1 = model.top_k_acc(logits, labels, k=1)
            batch_top3 = model.top_k_acc(logits, labels, k=3)

            epoch_loss += batch_loss.item()
            epoch_top1 += batch_top1
            epoch_top3 += batch_top3

            if mode == 'train':
                batch_loss.backward()
                optimizer.step()

        # sum of all batchs / num of batches
        epoch_loss /= i + 1 
        epoch_top1 /= i + 1
        epoch_top3 /= i + 1
        
        loss[mode] = epoch_loss
        top1[mode] = epoch_top1
        top3[mode] = epoch_top3
        
        print('epoch %d %s loss %.4f top1 %.4f top3 %.4f' % (epoch, mode, epoch_loss, epoch_top1, epoch_top3))
        # log to tensorboard
        if not (writer is None) and (mode == 'eval'):
            writer.add_scalars('%s_loss' % model.__class__.__name__,
                         tag_scalar_dict={'train': loss['train'], 
                                          'eval': loss['eval']}, 
                         global_step=epoch)
            writer.add_scalars('%s_top1' % model.__class__.__name__,
                         tag_scalar_dict={'train': top1['train'], 
                                          'eval': top1['eval']}, 
                         global_step=epoch)
            writer.add_scalars('%s_top3' % model.__class__.__name__,
                         tag_scalar_dict={'train': top3['train'], 
                                          'eval': top3['eval']}, 
                         global_step=epoch)
    for epoch in range(num_epoch):
        run_epoch('train')
        run_epoch('eval')

In [4]:
for n_hidden in [32, 64]:
    for optim_conf in [
        {'optim':'adam', 'lr':0.001},
        {'optim':'sgd', 'lr':0.1},
        {'optim':'sgd', 'lr':0.01}
    ]:
        model = CnnClassifier(n_hidden=n_hidden)
        if optim_conf['optim'] == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=optim_conf['lr'])
        else:
            optimizer = torch.optim.SGD(model.parameters(), lr=optim_conf['lr'])
        conf_str = str(n_hidden)+'_'+optim_conf['optim']+'_'+str(optim_conf['lr'])
        print(conf_str)
        train(
            model=model,
            loaders={
                'train': torch.utils.data.DataLoader(holdout_train_data, batch_size=32, shuffle=True),
                'eval': torch.utils.data.DataLoader(holdout_eval_data, batch_size=32, shuffle=True)
            },
            optimizer=optimizer, 
            writer=SummaryWriter('./logs/cnn_scratch/%s' % (conf_str)), 
            num_epoch=10, 
            device='cpu'
        )

32_adam_0.001
epoch 0 train loss 2.7049 top1 0.2475 top3 0.4809
epoch 0 eval loss 1.9404 top1 0.4255 top3 0.7171
epoch 1 train loss 1.6144 top1 0.5145 top3 0.7978
epoch 1 eval loss 1.3761 top1 0.5809 top3 0.8514
epoch 2 train loss 1.2455 top1 0.6189 top3 0.8689
epoch 2 eval loss 1.1413 top1 0.6457 top3 0.8865
epoch 3 train loss 1.0540 top1 0.6701 top3 0.9006
epoch 3 eval loss 1.0126 top1 0.6790 top3 0.9074
epoch 4 train loss 0.9481 top1 0.6980 top3 0.9158
epoch 4 eval loss 0.9374 top1 0.6997 top3 0.9179
epoch 5 train loss 0.8793 top1 0.7187 top3 0.9254
epoch 5 eval loss 0.9004 top1 0.7157 top3 0.9217
epoch 6 train loss 0.8359 top1 0.7310 top3 0.9316
epoch 6 eval loss 0.8352 top1 0.7311 top3 0.9311
epoch 7 train loss 0.7954 top1 0.7415 top3 0.9378
epoch 7 eval loss 0.8287 top1 0.7300 top3 0.9329
epoch 8 train loss 0.7651 top1 0.7505 top3 0.9420
epoch 8 eval loss 0.7804 top1 0.7492 top3 0.9373
epoch 9 train loss 0.7398 top1 0.7590 top3 0.9454
epoch 9 eval loss 0.7729 top1 0.7459 top3 0.9